In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import gseapy as gp
import numpy as np
import os

In [ ]:
org = gp.get_library_name(organism='Human')
org

In [ ]:
deg_file = '../deg/deg_results.xlsx'

# Load the Excel file
excel_file = pd.ExcelFile(deg_file)

# Get the sheet names
sheet_names = excel_file.sheet_names

# Print the sheet names
print(sheet_names)

In [ ]:
# query genes
# choose which pvalue column to use: pvalue or padj
# set the threshold for the pvalue

pval_col = 'padj'
threshold = 0.05

directory_path = f'{pval_col}_{threshold}'
if not os.path.exists(directory_path):
    # Directory does not exist, create it
    os.makedirs(directory_path)
    os.makedirs(f'{directory_path}/tables')
    os.makedirs(f'{directory_path}/figures')
    print(f"Directory '{directory_path}' created successfully.")

symbol_col = 'external_gene_name'

color_dict = {'GO_Molecular_Function_2021':'red', 'GO_Biological_Process_2021':'blue', 
                   'GO_Cellular_Component_2021':'green','KEGG_2021_Human':'yellow'}

for pval_col,threshold in zip(['padj'],[0.05]):

    for sheet_name in sheet_names:
        print(sheet_name)
        
        df = pd.read_excel(deg_file, sheet_name=sheet_name)
        df = df[df[symbol_col].notna()]
        
        # Load the list of differentially expressed genes
        diff_exp_genes = df[df[pval_col]<threshold][symbol_col].tolist()
        if len(diff_exp_genes)==0:
            print('No DEGs.')
            continue

        # Load the background gene set
        background_genes =  df[df[pval_col].notna()][symbol_col].tolist()

        # Set up the Enrichr query parameters
        enrichr_results = gp.enrichr(
            gene_list=diff_exp_genes,
            background=background_genes,
            organism='Human',
            gene_sets=['GO_Molecular_Function_2021', 'GO_Biological_Process_2021', 
                    'GO_Cellular_Component_2021','KEGG_2021_Human', 'Human_Gene_Atlas',
                    'Descartes_Cell_Types_and_Tissue_2021','ARCHS4_Tissues','GTEx_Tissue_Expression_Down',
                    'GTEx_Tissue_Expression_Up','GTEx_Tissues_V8_2023','Tissue_Protein_Expression_from_ProteomicsDB'
                    ],
            outdir='enrichr_results',
            cutoff=1
        )
            
        df = enrichr_results.results
        #filter enrichment results for adjusted p-value < 0.05
        df = df[df['Adjusted P-value']<0.05]
        if len(df)==0:
            print('No enrichment results.')
            continue
        df = df.sort_values(by='Adjusted P-value', ascending=False)
        
        #horizontal bar plot
        fig, ax = plt.subplots(figsize=(20, 10))
        gene_sets = sorted(list(set(df.Gene_set)))

        # create an ExcelWriter object to save results
        writer = pd.ExcelWriter(f'{pval_col}_{threshold}/tables/{sheet_name}.xlsx', engine='xlsxwriter')

        for gs in gene_sets:
            gs_df = df[df.Gene_set==gs]
            gs_df = gs_df.sort_values(by=['Adjusted P-value', 'Odds Ratio'], ascending=[True, False])
            gs_df = gs_df.reset_index(drop=True)
            
            if gs in ['GO_Molecular_Function_2021', 'GO_Biological_Process_2021', 
                    'GO_Cellular_Component_2021','KEGG_2021_Human']:
                # invert order for plotting 
                plot_gs_df = gs_df.copy()
                plot_gs_df = plot_gs_df.iloc[:5,:]
                plot_gs_df = plot_gs_df.iloc[::-1]
                ax.barh(y=plot_gs_df.Term, width=-np.log10(plot_gs_df['Adjusted P-value']), color=color_dict[gs], label=gs)

            #sheetname has to be shorter than 31 characters
            if len(gs) > 31:
                gs = gs[:31]
            gs_df.to_excel(writer, sheet_name=f'{gs}',index=False)
            
        ax.set_xlabel('-log10(adjusted p-value)', size=20)
        ax.yaxis.set_tick_params(labelsize=15)
        ax.xaxis.set_tick_params(labelsize=14)
        ax.set_title(sheet_name, size=24)
        ax.legend(loc='lower right', fontsize=15)
        plt.tight_layout()
        plt.savefig(f'{pval_col}_{threshold}/figures/{sheet_name}.png')
        
        # save the Excel file
        writer.close()

In [ ]:
#Allen Brain Up/Down - query only up or down genes
# choose which pvalue column to use: pvalue or padj
# set the threshold for the pvalue

pval_col = 'padj'
threshold = 0.05

directory_path = f'{pval_col}_{threshold}'
if not os.path.exists(f'{directory_path}/tables/brain/'):
    os.makedirs(f'{directory_path}/tables/brain/')
if not os.path.exists(f'{directory_path}/figures/brain/'):
    os.makedirs(f'{directory_path}/figures/brain/')

symbol_col = 'external_gene_name'

color_dict = {'Allen_Brain_Atlas_up':'red', 'Allen_Brain_Atlas_down':'blue'}

for pval_col,threshold in zip(['padj'],[0.05]):


    for sheet_name in sheet_names:
        print(sheet_name)
        
        df = pd.read_excel(deg_file, sheet_name=sheet_name)
        df = df[df[symbol_col].notna()]
        
        # Load the list of differentially expressed genes
        up_de_genes = df[(df[pval_col]<threshold) & (df.log2FoldChange>0)][symbol_col].tolist()
        down_de_genes = df[(df[pval_col]<threshold) & (df.log2FoldChange<0)][symbol_col].tolist()
        
        if len(up_de_genes)==0 and len(down_de_genes)==0:
            print('No DEGs.')
            continue

        # Load the background gene set
        background_genes =  df[df[pval_col].notna()][symbol_col].tolist()

        # Set up the Enrichr query parameters
        up_enrichr_results = gp.enrichr(
            gene_list=up_de_genes,
            background=background_genes,
            organism='Human',
            gene_sets=['Allen_Brain_Atlas_up'],
            outdir='enrichr_results',
            cutoff=1
        )
        
        down_enrichr_results = gp.enrichr(
            gene_list=down_de_genes,
            background=background_genes,
            organism='Human',
            gene_sets=['Allen_Brain_Atlas_down'],
            outdir='enrichr_results',
            cutoff=1
        )
            
        up_df = up_enrichr_results.results
        up_df = up_df[up_df['Adjusted P-value']<0.05]
        
        down_df = down_enrichr_results.results
        down_df = down_df[down_df['Adjusted P-value']<0.05]
        
        if up_df.shape[0]==0 and down_df.shape[0]==0:
            print('No enrichment results.')
            continue
        
        df = pd.concat([up_df, down_df])
        df.loc[df.Gene_set=='Allen_Brain_Atlas_up', 'color'] = 'red'
        df.loc[df.Gene_set=='Allen_Brain_Atlas_down', 'color'] = 'blue'
        df = df.sort_values(by='Adjusted P-value', ascending=False)

        #horizontal bar plot
        fig, ax = plt.subplots(figsize=(20, 10))
        gene_sets = sorted(list(set(df.Gene_set)))
        for gs in gene_sets:
            gs_df = df[df.Gene_set==gs]
            gs_df = gs_df.iloc[:5,:]
            ax.barh(y=gs_df.Term, width=-np.log10(gs_df['Adjusted P-value']), color=color_dict[gs], label=gs)
            
        ax.set_xlabel('-log10(adjusted p-value)', size=20)
        ax.yaxis.set_tick_params(labelsize=15)
        ax.xaxis.set_tick_params(labelsize=14)
        ax.set_title(sheet_name, size=24)
        ax.legend(loc='lower right', fontsize=15)
        plt.tight_layout()
        plt.savefig(f'{pval_col}_{threshold}/figures/brain/{sheet_name}.png')
        
        # create an ExcelWriter object
        writer = pd.ExcelWriter(f'{pval_col}_{threshold}/tables/brain/{sheet_name}.xlsx', engine='xlsxwriter')
        
        for gs in set(df.Gene_set):
            # write each data frame to a separate sheet
            gs_df = df[df.Gene_set==gs]
            if len(gs) > 31:
                gs = gs[:31]
            gs_df.to_excel(writer, sheet_name=f'{gs}',index=False)

        # save the Excel file
        writer.close()